# **Library Import**

In [ ]:

import numpy as np 
import pandas as pd 
import cv2



# **Data Set import**

In [ ]:
df = pd.read_csv('../input/facial-expression/fer2013.csv')

# **Preprocessing **

In [ ]:
df.head()

In [ ]:
df.iloc[0]['pixels']
print(df)
len(df.iloc[0]['pixels'].split())
# 48 * 48

In [ ]:
label_map = ['Anger', 'Neutral', 'Fear', 'Happy', 'Sad', 'Surprise']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
img = df.iloc[0]['pixels'].split()
# img                                    img is in string

In [ ]:
img = [int(i) for i in img]  #Conversion into integer

In [ ]:
type(img[0])

In [ ]:
len(img)

In [ ]:
img = np.array(img)

In [ ]:
img = img.reshape(48,48)

In [ ]:
img.shape

In [ ]:
plt.imshow(img, cmap='gray')
plt.xlabel(df.iloc[0]['emotion'])

In [ ]:
X = []     #List of images
y = []     #Emotions

# **label_map = ['Anger', 'Neutral', 'Fear', 'Happy', 'Sad', 'Surprise']**

In [ ]:
def getData(path):
    anger = 0
    fear = 0
    sad = 0
    happy = 0
    surprise = 0
    neutral = 0
    df = pd.read_csv(path)
    
    X = []
    y = []    
    
    for i in range(len(df)):
        if df.iloc[i]['emotion'] != 1:     #removal of disgust images
            if df.iloc[i]['emotion'] == 0:  #We will take 4000 images from each dataset [0] is anger emotion
                if anger <= 4000:            
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    anger += 1
                else:
                    pass
                
            if df.iloc[i]['emotion'] == 2:
                if fear <= 4000:            
                    y.append(df.iloc[i]['emotion'])# Fear
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    fear += 1
                else:
                    pass
                
            if df.iloc[i]['emotion'] == 3:
                if happy <= 4000:                  #fear
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    happy += 1
                else:
                    pass
                
            if df.iloc[i]['emotion'] == 4:
                if sad <= 4000:            
                    y.append(df.iloc[i]['emotion'])#Sad
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    sad += 1
                else:
                    pass
                
            if df.iloc[i]['emotion'] == 5:
                if surprise <= 4000:            #Surprise
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    surprise += 1
                else:
                    pass
                
            if df.iloc[i]['emotion'] == 6:
                if neutral <= 4000:            #neutral
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    neutral += 1
                else:
                    pass

            
            
    return X, y  
    

In [ ]:
X, y = getData('../input/facial-expression/fer2013.csv')

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
X = np.array(X)/255.0
y = np.array(y)

In [ ]:
X.shape, y.shape

In [ ]:
y_o = []
for i in y:
    if i != 6:
        y_o.append(i)
                                            #Moving 6th to another position 
    else:
        y_o.append(1)

In [ ]:
np.unique(y_o, return_counts=True)

In [ ]:
for i in range(5):
    r = np.random.randint((1), 24000, 1)[0]
    plt.figure()
    plt.imshow(X[r].reshape(48,48), cmap='gray')
    plt.xlabel(label_map[y_o[r]])

In [ ]:
X = X.reshape(len(X), 48, 48, 1)        #If color image 3 instead of 1 

In [ ]:
# no_of_images, height, width, coloar_map  In Cnn data should be in 4 dimensions 

In [ ]:
X.shape

In [ ]:
from keras.utils import to_categorical
y_new = to_categorical(y_o, num_classes=6)

In [ ]:
len(y_o), y_new.shape

In [ ]:
y_o[150], y_new[150]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

In [ ]:
model = Sequential()              #Exactly One input and one output (layer by layer modelling)


input_shape = (48,48,1)


model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model.add(Conv2D(64, (5, 5), padding='same'))
model.add(BatchNormalization())          #This has the effect of stabilizing the learning process and dramatically reducing
                                        #the number of training epochs required to train deep networks.
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model.add(Conv2D(128, (5, 5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

## (15, 15) --->  30
model.add(Flatten())
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

# fer_json = model.to_json()
# with open("fer.json", "w") as json_file:
#     json_file.write(fer_json)
# model.save_weights("fer.h5")

In [ ]:
history = model.fit(X, y_new, epochs=200, batch_size=64, shuffle=True, validation_split=0.2)
print(history.history.keys())





In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
# plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# model.save('model.h5')

In [ ]:
# fer_json = model.to_json()  
# with open("fer.json", "w") as json_file:  
#     json_file.write(fer_json)  
# model.save_weights("fer.h5")  

In [ ]:
# import cv2

In [ ]:
# test_img = cv2.imread('../input/fer2013/train/sad/Training_10022789.jpg', 0)

In [ ]:
# print (test_img)

In [ ]:
# test_img.shape

In [ ]:
# test_img = cv2.resize(test_img, (48,48))
# test_img.shape

In [ ]:
# test_img = test_img.reshape(1,48,48,1)

In [ ]:
# model.predict(test_img)

In [ ]:
# label_map = ['Anger', 'Neutral', 'Fear', 'Happy', 'Sad', 'Surprise']

In [ ]:
# import os  
# import cv2  
# import numpy as np  
# from keras.models import model_from_json  
# from keras.preprocessing import image  
  
# #load model  
# model = model_from_json(open("./fer.json", "r").read())  
# #load weights  
# model.load_weights('./fer.h5')  
  
  
# face_haar_cascade = cv2.CascadeClassifier('../input/harrcascade/haarcascade_frontalcatface.xml')  
  
  
# cap=cv2.VideoCapture(0)  
  
# while True:  
#     ret,test_img=cap.read()# captures frame and returns boolean value and captured image  
#     if not ret:  
#         continue  
#     gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  
  
#     faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)  
  
  
#     for (x,y,w,h) in faces_detected:  
#         cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)  
#         roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image  
#         roi_gray=cv2.resize(roi_gray,(48,48))  
#         img_pixels = image.img_to_array(roi_gray)  
#         img_pixels = np.expand_dims(img_pixels, axis = 0)  
#         img_pixels /= 255  
  
#         predictions = model.predict(img_pixels)  
  
#         #find max indexed array  
#         max_index = np.argmax(predictions[0])  
  
#         emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
#         predicted_emotion = emotions[max_index]  
  
#         cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  
  
#     resized_img = cv2.resize(test_img, (1000, 700))  
#     cv2.imshow('Facial emotion analysis ',resized_img)  
  
  
  
#     if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed  
#         break  
  
# cap.release()  
# cv2.destroyAllWindows  